### Creating Dataset and Training Model

In [1]:
Names = []

In [9]:
ID = input("Enter the id of user: ")
Name = input("Enter the name of user: ")

Enter the id of user: 2
Enter the name of user: Ishika


In [10]:
Names.append(Name)
print(Names)

['Aashi', 'Ishika']


In [11]:
import cv2
import numpy as np
import smtplib
import pywhatkit as kt
import boto3
import getpass as gp
import datetime
import os

In [12]:
# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [13]:
# Load functions
import cv2
import numpy as np
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face


<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-13-c9c8d796ffaf>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


In [14]:
# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

### Collecting samples 

In [15]:
# Collect samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = "./dataset/" + Name + "." + ID +"."+ str(count) + ".jpg"
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Cropped img', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

### Training Model

In [16]:
import cv2
import numpy as np
import os
from os import listdir
from os.path import isfile, join

def Image_and_id(path):
    image_paths = [os.path.join(path,f) for f in os.listdir(path)]
    facesamples, Ids = [], []
    
    for imagepath in image_paths:
        image = cv2.imread(imagepath, cv2.IMREAD_GRAYSCALE)
        imagenp = np.asarray(image,dtype=np.uint8)
        Id = int(os.path.split(imagepath)[-1].split(".")[1])
        face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        faces = face_classifier.detectMultiScale(imagenp, 1.3, 5)
        for (x,y,w,h) in faces:
            facesamples.append(imagenp[y:y+h, x:x+w])
            Ids.append(Id)
    return facesamples , Ids



model  = cv2.face.LBPHFaceRecognizer_create()
faces , Id = Image_and_id("./dataset/")
#Let's train our model 
model.train(faces, np.array(Id))
print("Model trained sucessefully")
model.save("./Trained_model.yml")

Model trained sucessefully


In [17]:
cap.release()

In [18]:
def email_sender(sender,password,receiver,msg):
    s=smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login(sender,password)
    s.sendmail(sender,receiver, msg)
    s.quit()

In [19]:

def whatapp_sender(phonenumber,msg):
    obj_now = datetime.datetime.now()
    kt.sendwhatmsg_instantly(phonenumber,msg,wait_time=15)
 #   kt.sendwhatmsg(phonenumber,msg,time_hour=obj_now.hour,time_min=obj_now.minute+1,wait_time=5)


In [20]:
import time

def launchec2Instance():
    client = boto3.client("ec2", region_name="ap-south-1")
    instances = client.run_instances(
        ImageId="ami-06a0b4e3b7eb7a300",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="awsinstancekey"
    )
    print("Instance launched",instances["Instances"][0]["InstanceId"])
    
    #for volume
    volume = client.create_volume(
        AvailabilityZone='ap-south-1a',
        Size=5,
        VolumeType= 'gp2'
    )
    volumeid=volume["VolumeId"]
    print("volume Created",volume["VolumeId"])

def  attach_volume():
    time.sleep(10)
    client = boto3.client("ec2", region_name="ap-south-1")
    response = client.attach_volume(
        Device='sdb',
        InstanceId=instances["Instances"][0]["InstanceId"],
        VolumeId=volume["VolumeId"],
    )
    print(response)

### Face recognition by ID

In [24]:
#face recognition
import cv2
import numpy as np
import os

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi

launched = False
sent = False

# Open Webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        results = model.predict(face)
        id = results[0]
        
        if results[1] < 500:
            conf = int(100*(1- results[1]/400))
            st = str(conf) + '% confidence of ' + Names[id-1]
            cv2.putText(image, st, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
            cv2.imshow('Face_Recognition', image ) 
            
        if conf > 88 and id == 1 and launched == False:
            cv2.imshow('Face_Recognition', image )  
            print("launched",conf)
            launchec2Instance()
            #break
            launched = True
            
            
        elif conf > 88 and id == 2 and sent == False:
            cv2.imshow('Face_Recognition', image )  
            email_sender("facerecognition963@gmail.com","[123abc]","shivanimandloi122324@gmail.com", Names[id-1]+" detected ")
            whatapp_sender("+919021837700", str(Names[id-1]) +" detected ")
            #break
            sent = True
            
        else:
            pass

            
    except:
        cv2.putText(image, "No Face Found", (50, 50) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face_Recognition', image )
        pass
    
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break    
                    
        
cap.release()
cv2.destroyAllWindows()

<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-24-4be7664b3575>:13: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


launched 89
Instance launched i-0615de71174d1dd3f
volume Created vol-0a9a05bf3b6d4299a


In [ ]:
cap.release()
cv2.destroyAllWindows()